In [1]:
# %%time
# # set up a local dask cluster for parallelisation and accessing the dashboard
# from dask.distributed import Client, LocalCluster
# cluster = LocalCluster()  # Launches a scheduler and workers locally
# client = Client(cluster)  # Connect to distributed cluster and override default
# # client = Client(address="127.0.0.1:8787")
# client

CPU times: total: 359 ms
Wall time: 6.11 s


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56416,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.73 GiB
Comm: tcp://127.0.0.1:56441,Total threads: 2
Dashboard: http://127.0.0.1:56448/status,Memory: 3.93 GiB
Nanny: tcp://127.0.0.1:56419,


In [8]:
%%time
from Dataprocessing import crop_netCDF, ERA5_hourly_daily, prep_hindcast
import xarray as xr
from pathlib import Path

CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
%%time
crop_netCDF(r"Z:\Massendaten\ERA5",47,55,5,15,globpat="*1996*.nc",
            to_netcdf=True)

CPU times: total: 328 ms
Wall time: 16.5 s


In [8]:
%time ERA5_hourly_daily("Z:\Massendaten\ERA5\Cropped", globpat="ERA5*1995_box.nc",to_netcdf=True)

CPU times: total: 1.86 s
Wall time: 12.7 s


In [4]:
%%time
hc = prep_hindcast(r"Z:\Massendaten\ECMWF\SEAS5_Box", "system_5*.nc")

CPU times: total: 1.53 s
Wall time: 2.33 s


In [9]:
hc.coords

Coordinates:
  * longitude  (longitude) float32 5.0 6.0 7.0 8.0 9.0 ... 12.0 13.0 14.0 15.0
  * latitude   (latitude) float32 55.0 54.0 53.0 52.0 51.0 50.0 49.0 48.0 47.0
  * member     (member) int32 0 1 2 3 4 5 6 7 8 9 ... 16 17 18 19 20 21 22 23 24
  * lead       (lead) int32 1 2 3 4 5 6 7 8 ... 208 209 210 211 212 213 214 215
  * init       (init) datetime64[ns] 1994-01-01 1994-02-01 ... 2016-12-01

Now I concatenate all ERA5 cropped and aggregated to daily pr data into one
xr.DataSet and save to netCDF for reloading into climpred HindcastEnsemble
without the need to use open_mfdataset(), which automatically chunks the data
 oveer dimension time. This is because apparently there is some issue with
 reducing over dim = "init" with climpred (not sure why).

In [9]:
%%time
path = Path(r"Z:\Massendaten\ERA5\Cropped\Daily\Regrid")
ERA5_all = xr.open_mfdataset(path.glob("ERA5*.nc"))
ERA5_all.to_netcdf(path / "Observations.nc")

CPU times: total: 234 ms
Wall time: 883 ms
